In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from dataset import pro_encoder, na_encoder, hhblits_encoder
from model import *
import pandas as pd

In [2]:
epochs = 500
batch_size = 64
root_dir = "./"
data_csv = "final_rna.csv"
hhblits_dir = "new/"
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [3]:
## random_split
# df = pd.read_csv(data_csv)
# df = df.sample(frac=1.0)
# df.to_csv('random_rna.csv')
# row = df.shape[0]
# print(row)
# idx_tr = round(row*0.8);idx_ts = round(row*0.1)
# num_spl = [idx_tr,idx_tr+idx_ts,idx_tr+2*idx_ts]
# df.iloc[0:num_spl[0],:].to_csv('train.csv',index=False)
# df.iloc[num_spl[0]:num_spl[1],:].to_csv('val.csv',index=False)
# df.iloc[num_spl[1]:num_spl[2],:].to_csv('test.csv',index=False)

In [4]:
dataset = []
for i in ['train','val','test']:
    filename = i+'.csv'
    df = pd.read_csv(filename)
    pro_ori = pro_encoder(df,'origin'); pro_ori = torch.tensor(pro_ori, dtype=torch.float)
    pro_mut = pro_encoder(df,'mutant'); pro_mut = torch.tensor(pro_mut, dtype=torch.float)
    na = na_encoder(df); na = torch.tensor(na, dtype=torch.float)
    hhb_ori, hhb_mut = hhblits_encoder(hhblits_dir,df)
    hhb_ori, hhb_mut = torch.tensor(hhb_ori, dtype=torch.float), torch.tensor(hhb_mut, dtype=torch.float)
    y = df['ddG(kcal/mol)'].values.tolist(); y = torch.tensor(y, dtype=torch.float)
    dataset.append(TensorDataset(pro_ori, pro_mut, hhb_ori, hhb_mut, na, y))

train_dataset = dataset[0]; val_dataset = dataset[1]; test_dataset = dataset[2]

In [5]:
print(len(train_dataset),len(val_dataset), len(test_dataset))
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

900 112 112


# MLP Network

In [6]:
# from torch.optim import Adam
# import numpy as np
# import math

# loss_fn = nn.MSELoss()
# MLPRegressor = MLPRegressor().to(device)

# optimizer = Adam(MLPRegressor.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
# MLPRegressor

In [7]:
# def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
#     min_val_loss = float("inf")
#     log = []
#     for epoch in range(1, epochs+1):
#         total_loss = 0.
#         best_model = None
#         model.train()
#         for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
#             optimizer.zero_grad()
#             X_ori = X_ori.to(device)
#             X_mut = X_mut.to(device)
#             X_muthhb = X_muthhb.to(device)
#             X_orihhb = X_orihhb.to(device)
#             X_na = X_na.to(device)
#             y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#             y = y.to(device)
#             loss = loss_fn(y_preds.ravel(), y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()*len(X_ori)
#         total_mse = total_loss/len(train_dataset)

#         # validate
#         model.eval()
#         val_losses = 0.
#         with torch.no_grad():
#             for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
#                 X_ori = X_ori.to(device)
#                 X_mut = X_mut.to(device)
#                 X_muthhb = X_muthhb.to(device)
#                 X_orihhb = X_orihhb.to(device)
#                 X_na = X_na.to(device)
#                 y = y.to(device)
#                 preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#                 val_loss = loss_fn(preds.ravel(), y)
#                 val_losses += val_loss.item()*len(X_ori)
#         val_mse = val_losses/len(val_dataset)

#         log.append([epoch,total_mse,val_mse])
#         print('| epoch {:3d} | lr {:02.5f} '
#                       'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
#                       format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
#         if (epoch >= 50) and (val_mse < min_val_loss):
#             min_val_loss = val_loss
#             best_model = model
#             torch.save(best_model.state_dict(), './result/mlp_best_all.pth')
#         scheduler.step()
#     f = open('./result/mlp_log.txt','w')
#     for i in log:
#         f.write(str(i)+'\n')
#     f.close()
# TrainModel(MLPRegressor, loss_fn, optimizer, train_loader, val_loader, epochs)

In [8]:
# test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

# test_mse = 0.
# test_result = []
# model = MLPRegressor.to(device)
# model.load_state_dict(torch.load('./result/mlp_best_all.pth'))
# model.eval()
# for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
#     X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
#     X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
#     X_na = X_na.to(device)
#     y = y.to(device)
#     preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#     loss = loss_fn(preds.ravel(), y)
    
#     test_result.append(y.tolist())
#     test_result.append(preds.tolist())
#     test_result.append(loss.tolist())
#     test_mse += loss.item()*len(X_ori)
# test_mse = test_mse/len(test_dataset)
# test_rmse = math.sqrt(test_mse)

# file_name = 'result/mlp_test_'+'result.txt'
# f = open(file_name,'w')
# for i in test_result:
#     f.write(str(i)+'\n')
# f.close()
# print(test_rmse)

# CNN Network

In [9]:
# from torch.optim import Adam
# import numpy as np
# import math

# loss_fn = nn.MSELoss()
# CNNRegressor = CNNRegressor().to(device)

# optimizer = Adam(CNNRegressor.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
# CNNRegressor

In [10]:
# def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
#     min_val_loss = float("inf")
#     log = []
#     for epoch in range(1, epochs+1):
#         total_loss = 0.
#         best_model = None
#         model.train()
#         for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
#             optimizer.zero_grad()
#             X_ori = X_ori.to(device)
#             X_mut = X_mut.to(device)
#             X_muthhb = X_muthhb.to(device)
#             X_orihhb = X_orihhb.to(device)
#             X_na = X_na.to(device)
#             y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#             y = y.to(device)
#             loss = loss_fn(y_preds.ravel(), y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()*len(X_ori)
#         total_mse = total_loss/len(train_dataset)

#         # validate
#         model.eval()
#         val_losses = 0.
#         with torch.no_grad():
#             for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
#                 X_ori = X_ori.to(device)
#                 X_mut = X_mut.to(device)
#                 X_muthhb = X_muthhb.to(device)
#                 X_orihhb = X_orihhb.to(device)
#                 X_na = X_na.to(device)
#                 y = y.to(device)
#                 preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#                 val_loss = loss_fn(preds.ravel(), y)
#                 val_losses += val_loss.item()*len(X_ori)
#         val_mse = val_losses/len(val_dataset)

#         log.append([epoch,total_mse,val_mse])
#         print('| epoch {:3d} | lr {:02.5f} '
#                       'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
#                       format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
#         if (epoch >= 50) and (val_mse < min_val_loss):
#             min_val_loss = val_loss
#             best_model = model
#             torch.save(best_model.state_dict(), './result/cnn_best_all.pth')
#         scheduler.step()
#     f = open('./result/cnn_log.txt','w')
#     for i in log:
#         f.write(str(i)+'\n')
#     f.close()
# TrainModel(CNNRegressor, loss_fn, optimizer, train_loader, val_loader, epochs)

In [11]:
# test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

# test_mse = 0.
# test_result = []
# model = CNNRegressor.to(device)
# model.load_state_dict(torch.load('result/cnn_best_all.pth'))
# model.eval()
# for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
#     X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
#     X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
#     X_na = X_na.to(device)
#     y = y.to(device)
#     preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
#     loss = loss_fn(preds.ravel(), y)
    
#     test_result.append(y.tolist())
#     test_result.append(preds.tolist())
#     test_result.append(loss.tolist())
#     test_mse += loss.item()*len(X_ori)
# test_mse = test_mse/len(test_dataset)
# test_rmse = math.sqrt(test_mse)

# file_name = 'result/cnn_test_result.txt'
# f = open(file_name,'w')
# for i in test_result:
#     f.write(str(i)+'\n')
# f.close()
# print(test_rmse)

# LSTM Network

In [12]:
# from torch.optim import Adam
# import numpy as np
# import math

# loss_fn = nn.MSELoss()
# LSTMRegressor = LSTMRegressor().to(device)

# optimizer = Adam(LSTMRegressor.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
# LSTMRegressor

In [13]:
# def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
#     min_val_loss = float("inf")
#     log = []
#     for epoch in range(1, epochs+1):
#         total_loss = 0.
#         best_model = None
#         model.train()
#         for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
#             optimizer.zero_grad()
#             X_ori = X_ori.to(device)
#             X_mut = X_mut.to(device)
#             X_muthhb = X_muthhb.to(device)
#             X_orihhb = X_orihhb.to(device)
#             X_na = X_na.to(device)
#             y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
#             y = y.to(device)
#             loss = loss_fn(y_preds.ravel(), y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()*len(X_ori)
#         total_mse = total_loss/len(train_dataset)

#         # validate
#         model.eval()
#         val_losses = 0.
#         with torch.no_grad():
#             for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
#                 X_ori = X_ori.to(device)
#                 X_mut = X_mut.to(device)
#                 X_muthhb = X_muthhb.to(device)
#                 X_orihhb = X_orihhb.to(device)
#                 X_na = X_na.to(device)
#                 y = y.to(device)
#                 preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
#                 val_loss = loss_fn(preds.ravel(), y)
#                 val_losses += val_loss.item()*len(X_ori)
#         val_mse = val_losses/len(val_dataset)

#         log.append([epoch,total_mse,val_mse])
#         print('| epoch {:3d} | lr {:02.5f} '
#                       'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
#                       format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
#         if (epoch >= 50) and (val_mse < min_val_loss):
#             min_val_loss = val_loss
#             best_model = model
#             torch.save(best_model.state_dict(), 'result/lstm_best_all.pth')
#         scheduler.step()
#     f = open('result/lstm_log.txt','w')
#     for i in log:
#         f.write(str(i)+'\n')
#     f.close()
# TrainModel(LSTMRegressor, loss_fn, optimizer, train_loader, val_loader, epochs)

In [14]:
# test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

# test_mse = 0.
# test_result = []
# model = LSTMRegressor.to(device)
# model.load_state_dict(torch.load('result/lstm_best_all.pth'))
# model.eval()
# for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
#     X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
#     X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
#     X_na = X_na.to(device)
#     y = y.to(device)
#     preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
#     loss = loss_fn(preds.ravel(), y)
    
#     test_result.append(y.tolist())
#     test_result.append(preds.tolist())
#     test_result.append(loss.tolist())
#     test_mse += loss.item()*len(X_ori)
# test_mse = test_mse/len(test_dataset)
# test_rmse = math.sqrt(test_mse)

# file_name = 'result/lstm_test_result.txt'
# f = open(file_name,'w')
# for i in test_result:
#     f.write(str(i)+'\n')
# f.close()
# print(test_rmse)

# BiLSTM Network

In [15]:
# from torch.optim import Adam
# import numpy as np
# import math

# loss_fn = nn.MSELoss()
# BLSTMRegressor = BLSTMRegressor().to(device)

# optimizer = Adam(BLSTMRegressor.parameters(), lr=1e-3)
# scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
# BLSTMRegressor

In [16]:
# def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
#     min_val_loss = float("inf")
#     log = []
#     for epoch in range(1, epochs+1):
#         total_loss = 0.
#         best_model = None
#         model.train()
#         for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
#             optimizer.zero_grad()
#             X_ori = X_ori.to(device)
#             X_mut = X_mut.to(device)
#             X_muthhb = X_muthhb.to(device)
#             X_orihhb = X_orihhb.to(device)
#             X_na = X_na.to(device)
#             y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
#             y = y.to(device)
#             loss = loss_fn(y_preds.ravel(), y)
#             loss.backward()
#             optimizer.step()
#             total_loss += loss.item()*len(X_ori)
#         total_mse = total_loss/len(train_dataset)

#         # validate
#         model.eval()
#         val_losses = 0.
#         with torch.no_grad():
#             for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
#                 X_ori = X_ori.to(device)
#                 X_mut = X_mut.to(device)
#                 X_muthhb = X_muthhb.to(device)
#                 X_orihhb = X_orihhb.to(device)
#                 X_na = X_na.to(device)
#                 y = y.to(device)
#                 preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
#                 val_loss = loss_fn(preds.ravel(), y)
#                 val_losses += val_loss.item()*len(X_ori)
#         val_mse = val_losses/len(val_dataset)

#         log.append([epoch,total_mse,val_mse])
#         print('| epoch {:3d} | lr {:02.5f} '
#                       'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
#                       format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
#         if (epoch >= 50) and (val_mse < min_val_loss):
#             min_val_loss = val_loss
#             best_model = model
#             torch.save(best_model.state_dict(), 'result/blstm_best_all.pth')
#         scheduler.step()
#     f = open('result/blstm_log.txt','w')
#     for i in log:
#         f.write(str(i)+'\n')
#     f.close()
# TrainModel(BLSTMRegressor, loss_fn, optimizer, train_loader, val_loader, epochs)

In [17]:
# test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

# test_mse = 0.
# test_result = []
# model = BLSTMRegressor.to(device)
# model.load_state_dict(torch.load('result/blstm_best_all.pth'))
# model.eval()
# for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
#     X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
#     X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
#     X_na = X_na.to(device)
#     y = y.to(device)
#     preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na, device)
#     loss = loss_fn(preds.ravel(), y)
    
#     test_result.append(y.tolist())
#     test_result.append(preds.tolist())
#     test_result.append(loss.tolist())
#     test_mse += loss.item()*len(X_ori)
# test_mse = test_mse/len(test_dataset)
# test_rmse = math.sqrt(test_mse)

# file_name = 'result/blstm_test_result.txt'
# f = open(file_name,'w')
# for i in test_result:
#     f.write(str(i)+'\n')
# f.close()
# print(test_rmse)

# Trans

In [18]:
from torch.optim import Adam
import numpy as np
import math

loss_fn = nn.MSELoss()
Trans = Trans().to(device)

optimizer = Adam(Trans.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50], gamma=0.1)
Trans

Trans(
  (transencoder1): TransformerEncoder(
    (embeddings): Embeddings(
      (token_embeddings): Linear(in_features=20, out_features=60, bias=True)
      (layer_norm): LayerNorm((60,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (layer_norm_1): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
        (layer_norm_2): LayerNorm((60,), eps=1e-05, elementwise_affine=True)
        (attention): MultiHeadAttention(
          (heads): ModuleList(
            (0): AttentionHead(
              (q): Linear(in_features=60, out_features=6, bias=True)
              (k): Linear(in_features=60, out_features=6, bias=True)
              (v): Linear(in_features=60, out_features=6, bias=True)
            )
            (1): AttentionHead(
              (q): Linear(in_features=60, out_features=6, bias=True)
              (k): Linear(in_features=60, out_features=6, bias=True)
          

In [19]:
def TrainModel(model, loss_fn, optimizer, train_loader, val_loader, epochs=epochs):
    min_val_loss = float("inf")
    log = []
    for epoch in range(1, epochs+1):
        total_loss = 0.
        best_model = None
        model.train()
        for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in train_loader:
            optimizer.zero_grad()
            X_ori = X_ori.to(device)
            X_mut = X_mut.to(device)
            X_muthhb = X_muthhb.to(device)
            X_orihhb = X_orihhb.to(device)
            X_na = X_na.to(device)
            y_preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
            y = y.to(device)
            loss = loss_fn(y_preds.ravel(), y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()*len(X_ori)
        total_mse = total_loss/len(train_dataset)

        # validate
        model.eval()
        val_losses = 0.
        with torch.no_grad():
            for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in val_loader:
                X_ori = X_ori.to(device)
                X_mut = X_mut.to(device)
                X_muthhb = X_muthhb.to(device)
                X_orihhb = X_orihhb.to(device)
                X_na = X_na.to(device)
                y = y.to(device)
                preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
                val_loss = loss_fn(preds.ravel(), y)
                val_losses += val_loss.item()*len(X_ori)
        val_mse = val_losses/len(val_dataset)

        log.append([epoch,total_mse,val_mse])
        print('| epoch {:3d} | lr {:02.5f} '
                      'train_loss {:5.5f} | val_loss {:5.5f} | val_rmse {:5.5f}'.
                      format(epoch, scheduler.get_last_lr()[0],total_mse, val_mse, math.sqrt(val_mse)))
        if (epoch >= 50) and (val_mse < min_val_loss):
            min_val_loss = val_loss
            best_model = model
            torch.save(best_model.state_dict(), 'result/trans_best_all.pth')
        scheduler.step()
    f = open('result/trans_log.txt','w')
    for i in log:
        f.write(str(i)+'\n')
    f.close()
TrainModel(Trans, loss_fn, optimizer, train_loader, val_loader, epochs)

| epoch   1 | lr 0.00100 train_loss 436.76062 | val_loss 2.42099 | val_rmse 1.55595
| epoch   2 | lr 0.00100 train_loss 2.24020 | val_loss 2.67586 | val_rmse 1.63580
| epoch   3 | lr 0.00100 train_loss 2.18292 | val_loss 2.29104 | val_rmse 1.51362
| epoch   4 | lr 0.00100 train_loss 2.25928 | val_loss 2.41600 | val_rmse 1.55435
| epoch   5 | lr 0.00100 train_loss 1.93946 | val_loss 2.12584 | val_rmse 1.45803
| epoch   6 | lr 0.00100 train_loss 1.96247 | val_loss 2.80894 | val_rmse 1.67599
| epoch   7 | lr 0.00100 train_loss 1.93121 | val_loss 2.31464 | val_rmse 1.52139
| epoch   8 | lr 0.00100 train_loss 1.69356 | val_loss 2.06610 | val_rmse 1.43739
| epoch   9 | lr 0.00100 train_loss 1.71400 | val_loss 2.29989 | val_rmse 1.51654
| epoch  10 | lr 0.00100 train_loss 1.54046 | val_loss 1.97603 | val_rmse 1.40571
| epoch  11 | lr 0.00100 train_loss 1.45072 | val_loss 2.10653 | val_rmse 1.45139
| epoch  12 | lr 0.00100 train_loss 1.41211 | val_loss 2.03643 | val_rmse 1.42703
| epoch  13 | 

In [ ]:
test_loader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size)

test_mse = 0.
test_result = []
model = Trans
model.load_state_dict(torch.load('result/trans_best_all.pth'))
model.eval()
for X_ori, X_mut, X_orihhb, X_muthhb, X_na, y in test_loader:
    X_mut = X_mut.to(device); X_muthhb = X_muthhb.to(device)
    X_ori = X_ori.to(device); X_orihhb = X_orihhb.to(device)
    X_na = X_na.to(device)
    y = y.to(device)
    preds = model(X_ori, X_mut, X_orihhb, X_muthhb, X_na)
    loss = loss_fn(preds.ravel(), y)
    
    test_result.append(y.tolist())
    test_result.append(preds.tolist())
    test_result.append(loss.tolist())
    test_mse += loss.item()*len(X_ori)
test_mse = test_mse/len(test_dataset)
test_rmse = math.sqrt(test_mse)

file_name = 'result/trans_test_result.txt'
f = open(file_name,'w')
for i in test_result:
    f.write(str(i)+'\n')
f.close()
print(test_rmse)

1.028423179456882
